In [1]:
import pandas as pd
import numpy as np

In [2]:
error_df = []
for i in range(1, 13):
    df = pd.read_csv('./data/2019{}-citibike-tripdata.csv'.format(str(i).zfill(2)))
    is_NaN = df.isnull()
    row_has_NaN = is_NaN.any(axis=1)
    rows_with_NaN = df[row_has_NaN]
    error_df.append(rows_with_NaN)
error_df = pd.concat(error_df, ignore_index=True)

In [3]:
error_df

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,2358,2019-01-08 13:48:13.8120,2019-01-08 14:27:32.0160,NaN,NaN,40.854,-73.890,NaN,NaN,40.857,-73.881,34550,Subscriber,1992,1
1,3358,2019-01-08 14:29:31.5150,2019-01-08 15:25:29.7130,NaN,NaN,40.857,-73.884,NaN,NaN,40.857,-73.884,34550,Subscriber,1992,1
2,596,2019-01-10 14:35:39.2120,2019-01-10 14:45:35.4130,NaN,NaN,40.863,-73.890,NaN,NaN,40.866,-73.884,34550,Subscriber,1986,1
3,816,2019-01-10 14:56:38.4250,2019-01-10 15:10:14.8120,NaN,NaN,40.857,-73.890,NaN,NaN,40.854,-73.902,34447,Subscriber,1986,1
4,393,2019-01-12 12:36:01.1440,2019-01-12 12:42:34.9520,NaN,NaN,40.854,-73.890,NaN,NaN,40.857,-73.884,34536,Customer,1997,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,419,2019-08-28 04:56:24.0760,2019-08-28 05:03:23.5780,NaN,NaN,40.860,-73.899,NaN,NaN,40.860,-73.887,34428,Customer,1998,1
176,856,2019-08-28 05:05:22.3540,2019-08-28 05:19:39.2880,NaN,NaN,40.860,-73.887,NaN,NaN,40.848,-73.908,34428,Customer,1998,1
177,949,2019-08-28 05:23:29.6880,2019-08-28 05:39:19.6870,NaN,NaN,40.848,-73.908,NaN,NaN,40.854,-73.902,34428,Customer,1998,1
178,739,2019-08-28 06:09:32.5250,2019-08-28 06:21:52.2510,NaN,NaN,40.854,-73.899,NaN,NaN,40.848,-73.902,34428,Customer,1998,1


In [4]:
location_df = []
for i in range(1, 13):
    df = pd.read_csv('./data/2019{}-citibike-tripdata.csv'.format(str(i).zfill(2)))
    df = df[df[['start station name', 'end station name']].notnull().all(1)]
    tmp_df = df[['start station id','start station name', 'start station latitude','start station longitude']].drop_duplicates()
    location_df.append(tmp_df)
location_df = pd.concat(location_df, ignore_index=True)

In [5]:
location_df = location_df.drop_duplicates()
location_df

,start station id,start station name,start station latitude,start station longitude
0,3160.0,Central Park West & W 76 St,40.778968,-73.973747
1,519.0,Pershing Square North,40.751873,-73.977706
2,3171.0,Amsterdam Ave & W 82 St,40.785247,-73.976673
3,504.0,1 Ave & E 16 St,40.732219,-73.981656
4,229.0,Great Jones St,40.727434,-73.993790
...,...,...,...,...
9696,3305.0,E 91 St & 2 Ave,40.782000,-73.950000
9697,3142.0,1 Ave & E 62 St,40.761000,-73.962000
9698,3110.0,Meserole Ave & Manhattan Ave,40.728000,-73.953000
9699,3854.0,Morgan Ave & Maspeth Ave,40.716000,-73.935000


# 尝试一：通过校准 Station Name 来区别站点，解决一个站点名对应多个站点ID & 坐标的情况。
# 校准量大 (放弃)

In [6]:
location_df.loc[location_df.duplicated(subset=['start station name'],keep=False)].sort_values(['start station name'])

,start station id,start station name,start station latitude,start station longitude
9649,504.0,1 Ave & E 16 St,40.731000,-73.983000
3,504.0,1 Ave & E 16 St,40.732219,-73.981656
401,3142.0,1 Ave & E 62 St,40.761227,-73.960940
9697,3142.0,1 Ave & E 62 St,40.761000,-73.962000
9673,478.0,11 Ave & W 41 St,40.761000,-73.998000
...,...,...,...,...
580,422.0,W 59 St & 10 Ave,40.770513,-73.988038
9695,3175.0,W 70 St & Amsterdam Ave,40.776000,-73.983000
49,3175.0,W 70 St & Amsterdam Ave,40.777480,-73.982886
667,335.0,Washington Pl & Broadway,40.729039,-73.994046


# 尝试二：通过校准 Station ID 来区别站点，解决一个站点 ID 对应多个站点名 & 坐标的情况。
# 校准量大 (放弃)

In [7]:
location_df.loc[location_df.duplicated(subset=['start station id'],keep=False)].sort_values(['start station id'])

,start station id,start station name,start station latitude,start station longitude
9658,167.0,E 39 St & 3 Ave,40.749000,-73.977000
436,167.0,E 39 St & 3 Ave,40.748901,-73.976049
3099,243.0,Fulton St & Rockwell Pl,40.688410,-73.979458
272,243.0,Fulton St & Rockwell Pl,40.688226,-73.979382
344,251.0,Mott St & Prince St,40.723180,-73.994800
...,...,...,...,...
9700,3759.0,Irving Ave & Jefferson St,40.704000,-73.926000
6406,3809.0,W 55 St & 6 Ave,40.763189,-73.978434
6249,3809.0,W 55 St & 6 Ave,40.763201,-73.978458
9699,3854.0,Morgan Ave & Maspeth Ave,40.716000,-73.935000


# 尝试三：通过校准 Station (lat, lon) 来区别站点，解决一个站点坐标对应多个站点ID & 站点名的情况。
# 校准量不大，可以逐条校准，作为目前的解决方案。

In [9]:
x = location_df.loc[location_df.duplicated(subset=['start station latitude','start station longitude'],keep=False)].sort_values(['start station latitude','start station longitude'])
x

,start station id,start station name,start station latitude,start station longitude
166,3714.0,Division Av & Hooper St,40.706842,-73.954435
8830,3714.0,Division Ave & Hooper St,40.706842,-73.954435
343,3701.0,Cliff St & Fulton St,40.708318,-74.005041
4132,3701.0,Cliff St & Fulton St (Old),40.708318,-74.005041
105,296.0,Division St & Bowery,40.714131,-73.997047
8887,296.0,Division St & Bowery (old),40.714131,-73.997047
185,3427.0,Lafayette St & Jersey St,40.724305,-73.996010
5532,3427.0,Lafayette St & Jersey St S,40.724305,-73.996010
43,3467.0,W Broadway & Spring Street,40.724947,-74.001659
7130,3467.0,W Broadway & Spring St,40.724947,-74.001659


# 探索发现，Pershing Square South 和 Pershing Square North 是两个独立的存在的站点，各自都存在数据。
# 因此可以判断从数据集内检索到的两条相关数据，是错误填写的数据，数据量较小可以直接删掉。

In [12]:
name = x.loc[9656]['start station name']
hah_df = []
for i in range(1, 13):
    df = pd.read_csv('./data/2019{}-citibike-tripdata.csv'.format(str(i).zfill(2)))
    df = df[df[['start station name', 'end station name']].notnull().all(1)]
    tmp_df = df.loc[ df['start station name'] == name]
    print(len(tmp_df))
    hah_df.append(tmp_df)
hah_df = pd.concat(hah_df, ignore_index=True)
hah_df

4234
3901
4806
5499
5494
4962
5284
6324
7244
7615
5486
3148


,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,112,2019-01-01 10:57:27.8790,2019-01-01 10:59:19.9080,517.0,Pershing Square South,40.751581,-73.97791,3235.0,E 41 St & Madison Ave,40.752165,-73.979922,15236,Subscriber,1994,1
1,1104,2019-01-01 11:30:58.5250,2019-01-01 11:49:22.8630,517.0,Pershing Square South,40.751581,-73.97791,3156.0,E 72 St & York Ave,40.766638,-73.953483,21391,Subscriber,1987,1
2,2900,2019-01-01 12:25:02.9810,2019-01-01 13:13:23.0100,517.0,Pershing Square South,40.751581,-73.97791,3164.0,Columbus Ave & W 72 St,40.777057,-73.978985,26527,Subscriber,1971,1
3,732,2019-01-01 12:55:15.7580,2019-01-01 13:07:28.1000,517.0,Pershing Square South,40.751581,-73.97791,3244.0,University Pl & E 8 St,40.731437,-73.994903,34281,Subscriber,1979,1
4,885,2019-01-01 13:16:07.6550,2019-01-01 13:30:53.5390,517.0,Pershing Square South,40.751581,-73.97791,403.0,E 2 St & 2 Ave,40.725029,-73.990697,27771,Subscriber,1952,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63992,873,2019-12-31 18:35:46.3500,2019-12-31 18:50:20.3480,517.0,Pershing Square South,40.751581,-73.97791,3686.0,Gansevoort St & Hudson St,40.739448,-74.005070,40378,Subscriber,1971,1
63993,1290,2019-12-31 18:42:31.3070,2019-12-31 19:04:01.8220,517.0,Pershing Square South,40.751581,-73.97791,3236.0,W 42 St & Dyer Ave,40.758985,-73.993800,20739,Customer,1968,1
63994,964,2019-12-31 19:16:58.8950,2019-12-31 19:33:03.7100,517.0,Pershing Square South,40.751581,-73.97791,339.0,Avenue D & E 12 St,40.725806,-73.974225,40538,Subscriber,1984,2
63995,1208,2019-12-31 20:02:51.3210,2019-12-31 20:22:59.9380,517.0,Pershing Square South,40.751581,-73.97791,3161.0,W 76 St & Columbus Ave,40.780184,-73.977285,16497,Subscriber,1990,1


In [13]:
name = x.loc[9686]['start station name']
hah_df = []
for i in range(1, 13):
    df = pd.read_csv('./data/2019{}-citibike-tripdata.csv'.format(str(i).zfill(2)))
    df = df[df[['start station name', 'end station name']].notnull().all(1)]
    tmp_df = df.loc[ df['start station name'] == name]
    print(len(tmp_df))
    hah_df.append(tmp_df)
hah_df = pd.concat(hah_df, ignore_index=True)
hah_df

8874
8084
10889
13625
14720
14778
16348
16564
16969
16426
11736
7562


,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,316,2019-01-01 00:04:43.7360,2019-01-01 00:10:00.6080,519.0,Pershing Square North,40.751873,-73.977706,518.0,E 39 St & 2 Ave,40.747804,-73.973442,32723,Subscriber,1964,1
1,1771,2019-01-01 00:58:32.2260,2019-01-01 01:28:03.8330,519.0,Pershing Square North,40.751873,-73.977706,3175.0,W 70 St & Amsterdam Ave,40.777480,-73.982886,35771,Customer,1988,1
2,1349,2019-01-01 03:51:52.7730,2019-01-01 04:14:22.0720,519.0,Pershing Square North,40.751873,-73.977706,3490.0,E 116 St & 2 Ave,40.796879,-73.937261,34860,Subscriber,1994,1
3,309,2019-01-01 06:09:25.5450,2019-01-01 06:14:35.5180,519.0,Pershing Square North,40.751873,-73.977706,3687.0,E 33 St & 1 Ave,40.743227,-73.974498,34741,Subscriber,1960,1
4,447,2019-01-01 07:12:20.5550,2019-01-01 07:19:47.7710,519.0,Pershing Square North,40.751873,-73.977706,492.0,W 33 St & 7 Ave,40.750200,-73.990931,35555,Subscriber,1969,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
156570,1445,2019-12-31 20:13:33.7130,2019-12-31 20:37:39.0530,519.0,Pershing Square North,40.751873,-73.977706,267.0,Broadway & W 36 St,40.750977,-73.987654,41645,Customer,1980,1
156571,1726,2019-12-31 22:40:03.9470,2019-12-31 23:08:50.0600,519.0,Pershing Square North,40.751873,-73.977706,3575.0,Crescent St & 34 Ave,40.761080,-73.930562,15289,Subscriber,1962,1
156572,1141,2019-12-31 23:00:10.6990,2019-12-31 23:19:12.4680,519.0,Pershing Square North,40.751873,-73.977706,411.0,E 6 St & Avenue D,40.722281,-73.976687,39911,Subscriber,1979,2
156573,245,2019-12-31 23:33:07.5980,2019-12-31 23:37:12.6920,519.0,Pershing Square North,40.751873,-73.977706,167.0,E 39 St & 3 Ave,40.748901,-73.976049,16115,Subscriber,1987,1


In [ ]:
lat = x.loc[9656]['start station latitude']
lon = x.loc[9656]['start station longitude']
hah_df = []
for i in range(1, 13):
    df = pd.read_csv('./data/2019{}-citibike-tripdata.csv'.format(str(i).zfill(2)))
    df = df[df[['start station name', 'end station name']].notnull().all(1)]
    tmp_df = df.loc[(df['start station latitude'] == lat) & (df['start station longitude'] == lon)]
    hah_df.append(tmp_df)
hah_df = pd.concat(hah_df, ignore_index=True)
hah_df

0
0
0
0
0
0
0
0
0
0
0
2


,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,378,2019-12-19 17:28:20.9660,2019-12-19 17:34:39.6840,517.0,Pershing Square South,40.752,-73.977,528.0,2 Ave & E 31 St,40.743,-73.977,34469,Subscriber,1963,1
1,829,2019-12-24 16:08:32.1090,2019-12-24 16:22:21.5410,519.0,Pershing Square North,40.752,-73.977,305.0,E 58 St & 3 Ave,40.761,-73.968,34469,Customer,1999,1


# 对 W Broadway & Spring Street 还有 W Broadway & Spring St 进行探索，发现是改名了

In [15]:
lat = x.loc[43]['start station latitude']
lon = x.loc[43]['start station longitude']
hah_df = []
for i in range(1, 13):
    df = pd.read_csv('./data/2019{}-citibike-tripdata.csv'.format(str(i).zfill(2)))
    df = df[df[['start station name', 'end station name']].notnull().all(1)]
    tmp_df = df.loc[(df['start station latitude'] == lat) & (df['start station longitude'] == lon)]
    print(len(tmp_df))
    hah_df.append(tmp_df)
hah_df = pd.concat(hah_df, ignore_index=True)
hah_df

1955
1963
3228
4398
4811
5802
5812
6032
6045
4886
3322
2035


,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,524,2019-01-01 00:32:14.4270,2019-01-01 00:40:59.2060,3467.0,W Broadway & Spring Street,40.724947,-74.001659,127.0,Barrow St & Hudson St,40.731724,-74.006744,34778,Subscriber,1963,2
1,736,2019-01-01 02:31:51.4250,2019-01-01 02:44:08.2220,3467.0,W Broadway & Spring Street,40.724947,-74.001659,546.0,E 30 St & Park Ave S,40.744449,-73.983035,18401,Subscriber,1994,1
2,1245,2019-01-01 08:20:23.9640,2019-01-01 08:41:09.4940,3467.0,W Broadway & Spring Street,40.724947,-74.001659,478.0,11 Ave & W 41 St,40.760301,-73.998842,26704,Subscriber,1977,1
3,2564,2019-01-01 08:59:01.7680,2019-01-01 09:41:46.2370,3467.0,W Broadway & Spring Street,40.724947,-74.001659,525.0,W 34 St & 11 Ave,40.755942,-74.002116,30773,Subscriber,1954,0
4,302,2019-01-01 10:39:00.7930,2019-01-01 10:44:03.0640,3467.0,W Broadway & Spring Street,40.724947,-74.001659,3690.0,Park Pl & Church St,40.713342,-74.009355,34756,Subscriber,1987,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50284,969,2019-12-31 18:12:55.1090,2019-12-31 18:29:04.2380,3467.0,W Broadway & Spring St,40.724947,-74.001659,3712.0,W 35 St & Dyer Ave,40.754692,-73.997402,28902,Subscriber,1971,1
50285,1355,2019-12-31 19:11:43.9300,2019-12-31 19:34:19.1210,3467.0,W Broadway & Spring St,40.724947,-74.001659,252.0,MacDougal St & Washington Sq,40.732264,-73.998522,40060,Subscriber,1991,1
50286,222,2019-12-31 20:02:14.4770,2019-12-31 20:05:57.3640,3467.0,W Broadway & Spring St,40.724947,-74.001659,161.0,LaGuardia Pl & W 3 St,40.729170,-73.998102,41939,Subscriber,1994,1
50287,470,2019-12-31 22:23:00.9080,2019-12-31 22:30:51.5110,3467.0,W Broadway & Spring St,40.724947,-74.001659,3686.0,Gansevoort St & Hudson St,40.739448,-74.005070,28489,Subscriber,1974,1
